In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import os
import datasets

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [5]:
cpt_path = "/workspace-SR006.nfs2/Bulatov_A/rmt/runs/test/4_by_4_mult/Llama-3.2-1B-Instruct/smol:qa1-5-1:9/SEGM_1x1024_1024_64_LR3e-04-lora-mnc-distill__short/checkpoint-5000/pytorch_model.bin"

model.load_state_dict(torch.load(cpt_path), strict=False)

<All keys matched successfully>

In [9]:
dataset_dir = "/workspace-SR006.nfs2/Bulatov_A/rmt/data/implicit_chain_of_thought/4_by_4_mult"

train_path = os.path.join(dataset_dir, "train")
valid_path = os.path.join(dataset_dir, "valid")

In [10]:
train_dataset = datasets.load_from_disk(train_path)
valid_dataset = datasets.load_from_disk(valid_path)

In [16]:
from torch.nn.utils.rnn import pad_sequence
id_pad_value = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id
def collate_fn(batch):
    input_ids = [torch.tensor(b['examples_all']) for b in batch]
    labels = [torch.tensor(b['labels_all']) for b in batch]
    attention_mask = [torch.ones_like(b, dtype=int) for b in input_ids]
    labels_mask = [torch.sign(torch.tensor(b['labels_all'])) for b in batch]


    input_ids = pad_sequence(input_ids, padding_value=id_pad_value, batch_first=True)
    labels = pad_sequence(labels, padding_value=id_pad_value, batch_first=True)
    attention_mask = pad_sequence(attention_mask, padding_value=0, batch_first=True)
    labels_mask = pad_sequence(labels_mask, padding_value=0, batch_first=True)

    collated = {'input_ids': input_ids,
                'labels': labels,
                'attention_mask': attention_mask,
                }
    # if args.num_mem_tokens is not None:
    #     # add labels mask only for RMT, ARMT
    #     collated['labels_mask'] = labels_mask.bool()
    return collated

In [116]:
def extract_cot(text):
    if '<|endoftext|>' not in text:
        return ''
    else:
        return text.split('<|endoftext|>')[0].strip()

def extract_answer(text):
    if '####' not in text:
        return ''
    else:
        ans = text.split('####')[-1]
        ans = ans.split('<|endoftext|>')[0]
        return ans.strip()
        
def compute_accuracy(eval_pred):
    preds = eval_pred.predictions[:, :-1]
    labels = eval_pred.label_ids[:, 1:]
    # inputs = eval_pred.inputs
    # losses = eval_pred.losses

    # labels = collated['labels'][:, 1:]

    labels_masks = labels > 0
    preds_full = [p[m] for p, m in zip(preds, labels_masks)]
    labels_full = [l[m] for l, m in zip(labels, labels_masks)]

    preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
    labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

    preds_cot = [extract_cot(p) for p in preds_full_text]
    preds_ans = [extract_answer(p) for p in preds_full_text]

    labels_cot = [extract_cot(l) for l in labels_full_text]
    labels_ans = [extract_answer(l) for l in labels_full_text]
    
    # Calculate accuracy only on the unignored tokens
    acc_cot = np.mean([c == l for c, l in zip(preds_cot, labels_cot)])
    acc_ans = np.mean([c == l for c, l in zip(preds_ans, labels_ans)])

    return {'accuracy_cot': acc_cot, 'accuracy_ans': acc_ans}

In [43]:
predictions = eval_pred.predictions
label_ids = eval_pred.label_ids
inputs = eval_pred.inputs
losses = eval_pred.losses
# elements = (self.predictions, self.label_ids)

NameError: name 'eval_pred' is not defined

In [17]:
batch = [valid_dataset[i] for i in range(10)]
collated = collate_fn(batch)

out = model(**collated)
print(out.keys())

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


odict_keys(['loss', 'logits', 'past_key_values'])


In [18]:
out.loss

tensor(1.3672e-06, grad_fn=<NllLossBackward0>)

In [24]:
preds = out.logits.argmax(dim=-1).cpu().numpy()
preds.shape

(10, 71)

In [47]:
preds_text = tokenizer.batch_decode(preds, add_special_tokens=True)

In [49]:
preds_text[0].split('<|endoftext|>')

[', 9 9 4 * 0 0 7 0 5 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 ',
 ' #### 5 5 6 0 8 2 0 1 ',
 ' #']

In [105]:
''.split('<|endoftext|>')[1]

IndexError: list index out of range

In [107]:
labels = collated['labels'][:, 1:]

labels_masks = labels > 0
preds_full = [p[m] for p, m in zip(preds[:, :-1], labels_masks)]
labels_full = [l[m] for l, m in zip(labels, labels_masks)]

preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

preds_cot = [extract_cot(p) for p in preds_full_text]
preds_ans = [extract_answer(p) for p in preds_full_text]

labels_cot = [extract_cot(l) for l in labels_full_text]
labels_ans = [extract_answer(l) for l in labels_full_text]

In [108]:
# labels = collated['labels'][:, 1:]

# labels_masks = labels > 0
# preds_full = [p[m] for p, m in zip(preds[:, :-1], labels_masks)]
# labels_full = [l[m] for l, m in zip(labels, labels_masks)]

# preds_full_text = tokenizer.batch_decode(preds_full, add_special_tokens=True)
# labels_full_text = tokenizer.batch_decode(labels_full, add_special_tokens=True)

# preds_cot = [p.split('<|endoftext|>')[0].strip() for p in preds_full_text]
# labels_cot = [l.split('<|endoftext|>')[0].strip() for l in labels_full_text]

# # preds_ans = [p.split('<|endoftext|>')[1].strip()[4:] for p in preds_full_text]
# # labels_ans = [l.split('<|endoftext|>')[1].strip()[4:] for l in labels_full_text]

# preds_ans = [p.split('####')[1].strip()[4:] for p in preds_full_text]
# labels_ans = [l.split('####')[1].split('astrip()[4:] for l in labels_full_text]



In [109]:
tokenizer.batch_decode(collated['input_ids'])

[' 5 6 3 2 * 7 4 3 4 <|endoftext|> 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|>',
 ' 6 9 1 5 * 6 4 4 7 <|endoftext|> 6 7 1 1 3 + 0 4 8 7 0 2 ( 6 1 0 9 3 2 ) + 0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 ) + 0 0 0 2 7 3 6 3 <|endoftext|> #### 6 1 4 9 8 6 8 3 <|endoftext|>',
 ' 6 7 3 9 * 8 9 1 7 <|endoftext|> 8 0 0 5 7 + 0 4 8 3 4 8 ( 8 4 8 8 1 9 ) + 0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 ) + 0 0 0 2 3 6 5 6 <|endoftext|> #### 8 4 4 8 8 4 7 6 <|endoftext|>',
 ' 3 0 3 4 * 3 4 6 5 <|endoftext|> 9 0 9 2 1 + 0 2 1 2 7 1 ( 9 2 0 5 8 1 ) + 0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 ) + 0 0 0 5 1 5 1 2 <|endoftext|> #### 9 2 8 1 8 2 4 2 <|endoftext|>',
 ' 0 3 3 7 * 8 5 6 5 <|endoftext|> 0 4 6 8 5 + 0 0 5 6 6 3 ( 0 4 1 5 2 4 ) + 0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 ) + 0 0 0 0 5 6 6 3 <|endoftext|> #### 0 4 1 3 7 4 1 4 <|endoftext|>',
 ' 3 6 0 6 * 4 3 8 7 <|endoftext|> 2 5 2 4 2 + 0 9 8 1 8 1 ( 2 4 1 6 0 2 ) + 0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 ) +

In [110]:
labels_full_text

[' 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|>',
 ' 6 7 1 1 3 + 0 4 8 7 0 2 ( 6 1 0 9 3 2 ) + 0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 ) + 0 0 0 2 7 3 6 3 <|endoftext|> #### 6 1 4 9 8 6 8 3 <|endoftext|>',
 ' 8 0 0 5 7 + 0 4 8 3 4 8 ( 8 4 8 8 1 9 ) + 0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 ) + 0 0 0 2 3 6 5 6 <|endoftext|> #### 8 4 4 8 8 4 7 6 <|endoftext|>',
 ' 9 0 9 2 1 + 0 2 1 2 7 1 ( 9 2 0 5 8 1 ) + 0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 ) + 0 0 0 5 1 5 1 2 <|endoftext|> #### 9 2 8 1 8 2 4 2 <|endoftext|>',
 ' 0 4 6 8 5 + 0 0 5 6 6 3 ( 0 4 1 5 2 4 ) + 0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 ) + 0 0 0 0 5 6 6 3 <|endoftext|> #### 0 4 1 3 7 4 1 4 <|endoftext|>',
 ' 2 5 2 4 2 + 0 9 8 1 8 1 ( 2 4 1 6 0 2 ) + 0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 ) + 0 0 0 1 4 4 2 4 <|endoftext|> #### 2 4 5 7 9 4 7 4 <|endoftext|>',
 ' 8 4 7 9 0 + 0 6 6 8 3 4 ( 8 0 4 8 4 4 ) + 0 0 4 7 8 4 0 ( 8 0 8 5 3 9 0 ) + 0 0 0 4 4 2 9 2 <|endoftext|> #### 8 0 8 9 7 

In [112]:
preds_cot

['5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0',
 '6 7 1 1 3 + 0 4 8 7 0 2 ( 6 1 0 9 3 2 ) + 0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 ) + 0 0 0 2 7 3 6 3',
 '8 0 0 5 7 + 0 4 8 3 4 8 ( 8 4 8 8 1 9 ) + 0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 ) + 0 0 0 2 3 6 5 6',
 '9 0 9 2 1 + 0 2 1 2 7 1 ( 9 2 0 5 8 1 ) + 0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 ) + 0 0 0 5 1 5 1 2',
 '0 4 6 8 5 + 0 0 5 6 6 3 ( 0 4 1 5 2 4 ) + 0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 ) + 0 0 0 0 5 6 6 3',
 '2 5 2 4 2 + 0 9 8 1 8 1 ( 2 4 1 6 0 2 ) + 0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 ) + 0 0 0 1 4 4 2 4',
 '8 4 7 9 0 + 0 6 6 8 3 4 ( 8 0 4 8 4 4 ) + 0 0 4 7 8 4 0 ( 8 0 8 5 3 9 0 ) + 0 0 0 4 4 2 9 2',
 '0 0 0 0 0 + 0 0 6 4 8 0 ( 0 0 6 4 8 0 ) + 0 0 2 9 6 1 0 ( 0 0 8 3 5 2 0 ) + 0 0 0 8 2 2 5 1',
 '5 5 5 2 2 + 0 6 6 0 7 2 ( 5 1 2 3 9 2 ) + 0 0 0 0 0 0 0 ( 5 1 2 3 9 2 0 ) + 0 0 0 5 5 5 2 2',
 '0 0 0 0 0 + 0 7 3 9 5 3 ( 0 7 3 9 5 3 ) + 0 0 9 7 9 1 1 ( 0 7 2 7 5 5 1 ) + 0 0 0 9 7 9 1 1']

In [113]:
labels_cot

['5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0',
 '6 7 1 1 3 + 0 4 8 7 0 2 ( 6 1 0 9 3 2 ) + 0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 ) + 0 0 0 2 7 3 6 3',
 '8 0 0 5 7 + 0 4 8 3 4 8 ( 8 4 8 8 1 9 ) + 0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 ) + 0 0 0 2 3 6 5 6',
 '9 0 9 2 1 + 0 2 1 2 7 1 ( 9 2 0 5 8 1 ) + 0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 ) + 0 0 0 5 1 5 1 2',
 '0 4 6 8 5 + 0 0 5 6 6 3 ( 0 4 1 5 2 4 ) + 0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 ) + 0 0 0 0 5 6 6 3',
 '2 5 2 4 2 + 0 9 8 1 8 1 ( 2 4 1 6 0 2 ) + 0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 ) + 0 0 0 1 4 4 2 4',
 '8 4 7 9 0 + 0 6 6 8 3 4 ( 8 0 4 8 4 4 ) + 0 0 4 7 8 4 0 ( 8 0 8 5 3 9 0 ) + 0 0 0 4 4 2 9 2',
 '0 0 0 0 0 + 0 0 6 4 8 0 ( 0 0 6 4 8 0 ) + 0 0 2 9 6 1 0 ( 0 0 8 3 5 2 0 ) + 0 0 0 8 2 2 5 1',
 '5 5 5 2 2 + 0 6 6 0 7 2 ( 5 1 2 3 9 2 ) + 0 0 0 0 0 0 0 ( 5 1 2 3 9 2 0 ) + 0 0 0 5 5 5 2 2',
 '0 0 0 0 0 + 0 7 3 9 5 3 ( 0 7 3 9 5 3 ) + 0 0 9 7 9 1 1 ( 0 7 2 7 5 5 1 ) + 0 0 0 9 7 9 1 1']

In [114]:
labels_ans

['5 5 6 0 8 2 0 1',
 '6 1 4 9 8 6 8 3',
 '8 4 4 8 8 4 7 6',
 '9 2 8 1 8 2 4 2',
 '0 4 1 3 7 4 1 4',
 '2 4 5 7 9 4 7 4',
 '8 0 8 9 7 1 0 3',
 '0 0 8 1 8 4 5 1',
 '5 1 2 8 4 8 2 2',
 '0 7 2 6 3 5 3 1']

In [115]:
preds_ans

['5 5 6 0 8 2 0 1',
 '6 1 4 9 8 6 8 3',
 '8 4 4 8 8 4 7 6',
 '9 2 8 1 8 2 4 2',
 '0 4 1 3 7 4 1 4',
 '2 4 5 7 9 4 7 4',
 '8 0 8 9 7 1 0 3',
 '0 0 8 1 8 4 5 1',
 '5 1 2 8 4 8 2 2',
 '0 7 2 6 3 5 3 1']

In [ ]:
labels_text
[0].split('<|endoftext|>')

NameError: name 'labels_texta' is not defined

In [41]:
for p, l, m in zip(preds, collated['labels'], collated['labels_mask']):
    print(p[m], l[m])

KeyError: 'labels_mask'

In [42]:
preds[0]

array([   11,   860,   860,   604,  1635,   657,   657,   767,   657,
         642,   642,   642,   642,   718,   352,  1343,   657,   657,
         718,   604,   860,   657,   357,   642,   642,   352,   352,
         352,   352,  1267,  1343,   657,   657,   642,   860,   657,
         767,   657,   357,   642,   642,   718,   657,   362,   807,
         657,  1267,  1343,   657,   657,   657,   657,   718,   604,
         860,   657,   220, 50256,  1303, 21017,   642,   642,   718,
         657,   807,   362,   657,   352,   220, 50256,  1303])

In [27]:
pred_texts = tokenizer.batch_decode(preds, skip_special_tokens=False)
print(pred_texts)

[', 9 9 4 * 0 0 7 0 5 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|> #', ' the 0 1 6 0 1 0 8 3 6 6 7 1 1 3 + 0 4 8 7 0 2 ( 6 1 0 9 3 2 ) + 0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 ) + 0 0 0 2 7 3 6 3 <|endoftext|> #### 6 1 4 9 8 6 8 3 <|endoftext|> #', ' the 0 8 4 + 2 8 3 0 8 8 0 0 5 7 + 0 4 8 3 4 8 ( 8 4 8 8 1 9 ) + 0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 ) + 0 0 0 2 3 6 5 6 <|endoftext|> #### 8 4 4 8 8 4 7 6 <|endoftext|> #', ', 2 3 2\n 0 0 0 1 9 9 0 9 2 1 + 0 2 1 2 7 1 ( 9 2 0 5 8 1 ) + 0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 ) + 0 0 0 5 1 5 1 2 <|endoftext|> #### 9 2 8 1 8 2 4 2 <|endoftext|> #', ' the 1 - 0 0 0 1 3 2 0 0 4 6 8 5 + 0 0 5 6 6 3 ( 0 4 1 5 2 4 ) + 0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 ) + 0 0 0 0 5 6 6 3 <|endoftext|> #### 0 4 1 3 7 4 1 4 <|endoftext|> #', ', 4 5 1 0 8 2 8 1 2 2 5 2 4 2 + 0 9 8 1 8 1 ( 2 4 1 6 0 2 ) + 0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 ) + 0 0 0 1 4 4 2 4 <|endoftext|> #### 2 4 5 7 9 4 7 4 <|endoftext|

In [28]:
labels_texts = tokenizer.batch_decode(collated['input_ids'], skip_special_tokens=False)
print(labels_texts)

[' 5 6 3 2 * 7 4 3 4 <|endoftext|> 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|>', ' 6 9 1 5 * 6 4 4 7 <|endoftext|> 6 7 1 1 3 + 0 4 8 7 0 2 ( 6 1 0 9 3 2 ) + 0 0 4 8 7 0 2 ( 6 1 4 7 1 3 2 ) + 0 0 0 2 7 3 6 3 <|endoftext|> #### 6 1 4 9 8 6 8 3 <|endoftext|>', ' 6 7 3 9 * 8 9 1 7 <|endoftext|> 8 0 0 5 7 + 0 4 8 3 4 8 ( 8 4 8 8 1 9 ) + 0 0 6 7 3 9 0 ( 8 4 4 6 5 8 1 ) + 0 0 0 2 3 6 5 6 <|endoftext|> #### 8 4 4 8 8 4 7 6 <|endoftext|>', ' 3 0 3 4 * 3 4 6 5 <|endoftext|> 9 0 9 2 1 + 0 2 1 2 7 1 ( 9 2 0 5 8 1 ) + 0 0 8 1 8 5 2 ( 9 2 8 6 6 7 2 ) + 0 0 0 5 1 5 1 2 <|endoftext|> #### 9 2 8 1 8 2 4 2 <|endoftext|>', ' 0 3 3 7 * 8 5 6 5 <|endoftext|> 0 4 6 8 5 + 0 0 5 6 6 3 ( 0 4 1 5 2 4 ) + 0 0 0 8 9 3 4 ( 0 4 1 3 2 8 4 ) + 0 0 0 0 5 6 6 3 <|endoftext|> #### 0 4 1 3 7 4 1 4 <|endoftext|>', ' 3 6 0 6 * 4 3 8 7 <|endoftext|> 2 5 2 4 2 + 0 9 8 1 8 1 ( 2 4 1 6 0 2 ) + 0 0 4 0 5 8 4 ( 2 4 5 6 5 0 5 ) + 0 0 

In [29]:
collated['input_ids'].shape

torch.Size([10, 71])

In [30]:
pred_texts[0]

', 9 9 4 * 0 0 7 0 5 5 5 5 6 1 + 0 0 6 4 9 0 ( 5 5 1 1 1 1 ) + 0 0 5 9 0 7 0 ( 5 5 6 0 2 8 0 ) + 0 0 0 0 6 4 9 0 <|endoftext|> #### 5 5 6 0 8 2 0 1 <|endoftext|> #'

In [33]:
preds[0]

array([   11,   860,   860,   604,  1635,   657,   657,   767,   657,
         642,   642,   642,   642,   718,   352,  1343,   657,   657,
         718,   604,   860,   657,   357,   642,   642,   352,   352,
         352,   352,  1267,  1343,   657,   657,   642,   860,   657,
         767,   657,   357,   642,   642,   718,   657,   362,   807,
         657,  1267,  1343,   657,   657,   657,   657,   718,   604,
         860,   657,   220, 50256,  1303, 21017,   642,   642,   718,
         657,   807,   362,   657,   352,   220, 50256,  1303])

In [ ]:
collated['input_ids'][0]

torch.Size([71])

In [38]:
for p, t in zip(preds[0], collated['input_ids'][0][1:]):
    # print(p, tokenizer.decode([p]), t, tokenizer.decode([t]))
    print(tokenizer.decode([p]), tokenizer.decode([t]))

,  6
 9  3
 9  2
 4  *
 *  7
 0  4
 0  3
 7  4
 0  
 5 <|endoftext|>
 5  5
 5  5
 5  5
 6  6
 1  1
 +  +
 0  0
 0  0
 6  6
 4  4
 9  9
 0  0
 (  (
 5  5
 5  5
 1  1
 1  1
 1  1
 1  1
 )  )
 +  +
 0  0
 0  0
 5  5
 9  9
 0  0
 7  7
 0  0
 (  (
 5  5
 5  5
 6  6
 0  0
 2  2
 8  8
 0  0
 )  )
 +  +
 0  0
 0  0
 0  0
 0  0
 6  6
 4  4
 9  9
 0  0
   
<|endoftext|> <|endoftext|>
 #  #
### ###
 5  5
 5  5
 6  6
 0  0
 8  8
 2  2
 0  0
 1  1
   
<|endoftext|> <|endoftext|>
